# explore gold and silver using FRED data

In [ ]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")
fred<-Fred()

In [ ]:
goldFred <- fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%gold%') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS)

print(data.frame(goldFred))

In [ ]:
silverFred <- fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%silver%') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS)

print(data.frame(silverFred))